In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-120285")
exp = Experiment(workspace=ws, name="project-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FM4BE5F6F to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-120285
Azure region: southcentralus
Subscription id: b3fb99ab-ce93-4e7d-92ea-a05123349c1b
Resource group: aml-quickstarts-120285


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Choose a name for your CPU cluster
# Use vm_size = \"Standard_D2_V2\" in your provisioning configuration.
cpu_cluster_name = "project-test"
vm_size = "Standard_D2_V2"
# max_nodes should be no greater than 4.

# Reference - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os

# Specify parameter sampler
# Reference - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

# Reference - scikit-learn
ps = RandomParameterSampling( {

        "--C": uniform(0.1, 0.9),
        "--max_iter": choice(25, 50, 75, 100)
    })

# Specify a Policy
# testing this instead of Bandit
# policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# Reference - https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py
est = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# Reference - https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name="Accuracy", estimator=est, policy=policy,
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=8)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hd_run = exp.submit(hyperdrive_config)

# Reference - https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets.rundetails?view=azure-ml-py
RunDetails(hd_run).show()
# below throws an error
#hd_run.wait_for_completion(show_output=True)



WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.


# best_run = hd_run.get_best_run_by_primary_metric()
# joblib.dump(best_run, 'hyperdrive_best_run.sav')

# The above was throwing a thread lock error. Uusing the below method instead to save it-
# Reference = https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run.run



best_run = hd_run.get_best_run_by_primary_metric()
best_model = best_run.register_model(model_name="project-1-hd", model_path="./")


In [14]:
# code from Classsroom
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print("Best Run Id: ", best_run.id)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Parameters: ", parameter_values["runDefinition"]["arguments"])


Best Run Id:  HD_85a01a87-5214-4484-b900-ebf4ba7b2cc0_2
Accuracy:  0.95
Parameters:  ['--C', '0.5401408285696929', '--max_iter', '50']


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###